In [41]:
import h5py
import pandas as pd
import numpy as np
import os, sys, tqdm

In [35]:
data_dir = '/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer_pipeline/enformer_predictions/kawakami-reference'

In [54]:
predictions_list = os.listdir(data_dir)
len(predictions_list), predictions_list[0:5]

(39954,
 ['neg_peak470228_predictions.h5',
  'neg_peak405343_predictions.h5',
  'pos_peak361823_predictions.h5',
  'pos_peak271667_predictions.h5',
  'pos_peak360679_predictions.h5'])

In [55]:
def agg_by_center(pred_tracks, center=8):

    y = []
    X = []

    for k, v in pred_tracks.items():
        y.append(1) if k.startswith('pos') else y.append(0)
        v = v[center, :]
        v = np.expand_dims(v, axis=1).T
        X.append(v)

    y = np.expand_dims(np.array(y), axis=1)
    dt = np.hstack((y, np.vstack(X)))

    return dt

In [56]:
training_predictions = {}

for dt in tqdm.tqdm(predictions_list):
    fle = f'{data_dir}/{dt}'
    if os.path.isfile(fle):
        with h5py.File(fle, 'r') as f:
            filekey = list(f.keys())[0]
            #print(filekey)
            # should I select tracks? ; maybe not yet
            #kk = 0 if dt.startswith('neg') else 1
            training_predictions[dt] = np.vstack(list(f[filekey]))
    else:
        print('File does not exist')

100%|██████████| 39954/39954 [35:24<00:00, 18.81it/s]  


In [57]:
full_dt = pd.DataFrame(agg_by_center(training_predictions))

In [59]:
column_names = ['class']
column_names.extend([f'f_{i}' for i in range(1, full_dt.shape[1])])
column_names[0:5] ; column_names[-5:-1]

['f_5309', 'f_5310', 'f_5311', 'f_5312']

In [61]:
ty = full_dt.set_axis(column_names, axis=1, inplace=False)
ty.iloc[0:5, -5:-1]
#

/tmp/ipykernel_22734/3853716067.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  ty = full_dt.set_axis(column_names, axis=1, inplace=False)


,f_5309,f_5310,f_5311,f_5312
0,0.000139,0.000153,0.000294,0.000231
1,0.002689,0.006143,0.006855,0.020428
2,0.002295,0.004101,0.003856,0.023600
3,0.001199,0.001549,0.001981,0.022603
4,0.000623,0.001229,0.002692,0.006986


In [63]:
save_dir = '/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/modeling_pipeline/data/train-test-val/kawakami/data_2022-12-12'
id = 'kawakami_aggByCenter_FOXA1_old'

In [64]:
ty.to_csv(path_or_buf=f'{save_dir}/{id}.csv.gz', index=False, compression='gzip')

In [67]:
ty['class'].value_counts()

0.0    20000
1.0    19954
Name: class, dtype: int64

In [ ]:
print(f'[INFO] Finished collecting {len(freedman_predictions)} predictions for {each_individual}')

dt_aggbycenter = agg_by_center(freedman_predictions, center=8)
data_list = [dt_aggbycenter]